# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
inputCSV = "outputData/cities.csv"
cityList = pd.read_csv(inputCSV)
print(cityList.columns)

#rename my column names to prevent probs later
cityList.rename(columns = {'Unnamed: 0':'Unamed','Max Temp (F)':'MaxTemp', 'Feels Like':'FeelsLike', 'Wind Speed (MPH)':'WindSpeed','Date and Time of Reading (UTC)':'DateTimeUTC'}, inplace = True)
print(cityList.columns)
del cityList['Unamed']
print(cityList.columns)
cityList.head()


Index(['Unnamed: 0', 'City', 'Country', 'Hemisphere', 'Latitude', 'Longitude',
       'Max Temp (F)', 'Humidity', 'Feels Like', 'Cloudiness',
       'Wind Speed (MPH)', 'Date and Time of Reading (UTC)'],
      dtype='object')
Index(['Unamed', 'City', 'Country', 'Hemisphere', 'Latitude', 'Longitude',
       'MaxTemp', 'Humidity', 'FeelsLike', 'Cloudiness', 'WindSpeed',
       'DateTimeUTC'],
      dtype='object')
Index(['City', 'Country', 'Hemisphere', 'Latitude', 'Longitude', 'MaxTemp',
       'Humidity', 'FeelsLike', 'Cloudiness', 'WindSpeed', 'DateTimeUTC'],
      dtype='object')


,City,Country,Hemisphere,Latitude,Longitude,MaxTemp,Humidity,FeelsLike,Cloudiness,WindSpeed,DateTimeUTC
0,Longyearbyen,SJ,Northern,78.22,15.64,32.00,94,17.87,40,18.34,2021-01-03 05:54:09
1,Qaanaaq,GL,Northern,77.48,-69.36,-2.83,76,-10.21,28,1.36,2021-01-03 05:52:21
2,Dikson,RU,Northern,73.51,80.55,7.11,96,-3.48,96,8.12,2021-01-03 05:52:31
3,Upernavik,GL,Northern,72.79,-56.15,8.31,91,-3.73,100,10.74,2021-01-03 05:51:28
4,Khatanga,RU,Northern,71.97,102.50,-24.07,79,-33.52,97,4.38,2021-01-03 05:46:11


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:


# Store latitude and longitude in locations
latLong = cityList[['Latitude', 'Longitude']]

# Fill NaN values and convert to float
humid = cityList["Humidity"]
latLong.head()

,Latitude,Longitude
0,78.22,15.64
1,77.48,-69.36
2,73.51,80.55
3,72.79,-56.15
4,71.97,102.50


In [5]:
# Plot Heatmap
mapLayout = {
    'width': '1050',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Use the gmaps.figure passing a zoom_level of 2 and a center point so the map displays properly at
# a readable size
myMap = gmaps.figure(layout=mapLayout,zoom_level=2,center=(15,25))
# Create heat layer
heatLayer = gmaps.heatmap_layer(latLong, weights=humid, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)

# Add heat layer
myMap.add_layer(heatLayer)

# Display figure
myMap

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# I like snow sports so temps are cold, but mostly sunny.  I'll stay in the Americas because COVID
# makes trans-oceanic travel extra difficult.
myTemp = (cityList.MaxTemp <32) & (cityList.FeelsLike > 0)
myWindSpeed = cityList.WindSpeed < 15
myCloudiness = cityList.Cloudiness < 40
# This keeps us in western hemisphere - had to use -20 because 0 allowed some spots in Spain and Portugal
myLongitude = cityList.Longitude < -20
myWeather = myTemp & myWindSpeed & myCloudiness & myLongitude

snowSportWeather = cityList[myWeather]
snowSportWeather.dropna()
snowSportWeather.reset_index(inplace = True, drop = True)
snowSportWeather

,City,Country,Hemisphere,Latitude,Longitude,MaxTemp,Humidity,FeelsLike,Cloudiness,WindSpeed,DateTimeUTC
0,Grand Forks,US,Northern,47.94,-97.43,28.40,92,14.92,1,8.05,2021-01-03 05:56:39
1,Miles City,US,Northern,46.41,-105.84,30.20,68,22.01,1,5.82,2021-01-03 05:57:48
2,Saint Anthony,US,Northern,45.02,-93.22,21.99,85,9.86,1,8.05,2021-01-03 05:54:47
3,Bar Harbor,US,Northern,44.39,-68.20,25.00,92,14.68,1,5.82,2021-01-03 05:56:54
4,Pierre,US,Northern,44.37,-100.35,26.60,79,18.43,1,5.82,2021-01-03 06:01:26
5,Show Low,US,Northern,34.25,-110.03,24.80,53,14.43,1,6.93,2021-01-03 06:02:30


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotelDF = snowSportWeather

hotelDF['HotelName'] = ''
hotelDF.head()

# params dictionary 
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}    

for index, row in hotelDF.iterrows():
    # get lat, lng from hotel_df
    lat = row["Latitude"]
    long = row["Longitude"]

    params["location"] = f"{lat},{long}"

    # Use the search term: "Hotel" and our lat/lng
    searchURL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    nameURL = requests.get(searchURL, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    nameURL = nameURL.json()
    #print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotelDF.loc[index, "HotelName"] = nameURL["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotelDF


<ipython-input-9-0b29e201b515>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotelDF['HotelName'] = ''
/Users/Kevin/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,City,Country,Hemisphere,Latitude,Longitude,MaxTemp,Humidity,FeelsLike,Cloudiness,WindSpeed,DateTimeUTC,HotelName
0,Grand Forks,US,Northern,47.94,-97.43,28.40,92,14.92,1,8.05,2021-01-03 05:56:39,TLF Temporary Lodging Facility
1,Miles City,US,Northern,46.41,-105.84,30.20,68,22.01,1,5.82,2021-01-03 05:57:48,Sleep Inn & Suites
2,Saint Anthony,US,Northern,45.02,-93.22,21.99,85,9.86,1,8.05,2021-01-03 05:54:47,"The Marquette Hotel, Curio Collection by Hilton"
3,Bar Harbor,US,Northern,44.39,-68.20,25.00,92,14.68,1,5.82,2021-01-03 05:56:54,Hampton Inn Bar Harbor
4,Pierre,US,Northern,44.37,-100.35,26.60,79,18.43,1,5.82,2021-01-03 06:01:26,Clubhouse Hotel & Suites
5,Show Low,US,Northern,34.25,-110.03,24.80,53,14.43,1,6.93,2021-01-03 06:02:30,Holiday Inn Express Show Low


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{HotelName}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotelInfo = [info_box_template.format(**row) for index, row in hotelDF.iterrows()]
hotelLocations = hotelDF[["Latitude", "Longitude"]]

In [15]:
# Add marker layer to heat map
map2Layout = {
    'width': '1050',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=map2Layout,zoom_level=4,center=(40,-100))

# Create hotel symbol layer

hotelLayer = gmaps.marker_layer(hotelLocations,hotelInfo)

# Add layer
fig.add_layer(heatLayer)
fig.add_layer(hotelLayer)
# Display map
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…